In [2]:
import os
#import psutil
import rasterio
from rasterio.features import rasterize
import numpy as np
import geopandas as gpd
import pandas as pd
from sklearn.feature_extraction import image
from shapely.geometry import Polygon
from shapely.ops import cascaded_union
from tqdm import tqdm
import json
import math
import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize
import matplotlib
import matplotlib.pyplot as plt
import cv2
import scipy.misc
from skimage.draw import line_aa
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon, MultiPolygon
from shapely.ops import cascaded_union
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import imageio

## Génération des grands masques depuis les fichiers GeoJSON

In [4]:
# A MODIFIER AVEC LES BONS CHEMINS POUR RUN LE CODE SUR VOTRE MACHINE
path_GeoTIFF = "/media/limbothai/LimboBooster/PIE_data/Airport_Feature_Auto_Detection/GeoTIFF/"
path_GeoJson = "/media/limbothai/LimboBooster/PIE_data/Airport_Feature_Auto_Detection/GeoJson/"

#image considérée
#names_map=["VHHH","LFPG","SBGR","CYUL","DNMM","VTBD","LFBO","OLBA","CYEG","KLAX","EIDW","ZBSJ","RKPC","LFOB","LIRP","LFMN","ZSDY","FMEE","EDDW","LFMP"]
names_map=["LFPG"]
for name_map in names_map:
    files=[]
    for i in os.listdir(path_GeoJson):
        if os.path.isfile(os.path.join(path_GeoJson,i)) and name_map in i:
            files.append(i)

    filepath = path_GeoJson+files[0]

    #chemin de sortie pour le masque png
    outfolder = "/media/limbothai/LimboBooster/PIE_data/Airport_Feature_Auto_Detection/Datasets/global_model/big_maskstest/"  
    os.makedirs(outfolder, exist_ok=True)

    # Liste des features d'intérêt et color mapping (codé sur 1 canal 8 bits, ie nuances de gris de 1 à 255, 0 est background)
    color_mapping = {
        'VerticalStructure.verticalpolygonalstructure': 1,
        'Taxiway.taxiwayshoulder':2,
        'Runway.runwayshoulder':2,
        'ParkingStandArea.parkingstandarea':3,
        'Runway.runwaydisplacedarea':3,
        'ServiceRoad.serviceroad':4,
        'Apron.apronelement':5,
        'Stop.stopway':6,
        'FinalApproachAndTakeOffArea.finalapproachandtakeoffarea':7,
        #0 correspond à Taxiway.taxiwayelement
        '0':8,
        'Runway.runwayelement':9,
        'Runway.runwayintersection':10,
        'TouchDownLiftOffArea.touchdownliftoffarea':11,
        'Runway.runwaymarking':12,
        'Taxiway.taxiwayholdingposition':13,
        'LandAndHoldShortOperationLocation.landandholdshortoperationlocation':13,
        'Runway.runwayexitline':14
    }

    #List of features to identify.
    #The order in this list matters as if there is superposition between 2 features or more,
    #the last coming in the list will dominates the others and give its color to the corresponding pixels. <3
    features_list = ['VerticalStructure.verticalpolygonalstructure',
        'Taxiway.taxiwayshoulder',
        'Runway.runwayshoulder',
        'ParkingStandArea.parkingstandarea',
        'Runway.runwaydisplacedarea',
        'ServiceRoad.serviceroad',
        'Apron.apronelement',
        'Stop.stopway',
        'FinalApproachAndTakeOffArea.finalapproachandtakeoffarea',
        #0 correspond à Taxiway.taxiwayelement
        '0',
        'Runway.runwayelement',
        'Runway.runwayintersection',
        'TouchDownLiftOffArea.touchdownliftoffarea',
        'Runway.runwaymarking']

    line_list = ['Taxiway.taxiwayholdingposition',
        'LandAndHoldShortOperationLocation.landandholdshortoperationlocation',
        'Runway.runwayexitline']

    def draw_linestring(line_list,whole_mask,transform):
        #transform = src.meta['transform']
        df = func_df(filepath)
        df = df[df['geometry'].geom_type=='LineString']
        for k in df.id.keys():
            for ls in line_list:
                if df['feattype'][k]['value']==ls:
                    coords = np.array(df['geometry'][k].coords)
                    for i in range(len(coords)-1):

                        c0,r0 = ~transform * tuple(coords[i])

                        c1,r1 = ~transform * tuple(coords[i+1])

                        rr, cc, val = line_aa(math.floor(r0),math.floor(c0),math.floor(r1), math.floor(c1))
                        new_val = np.ones(len(val))

                        whole_mask[rr, cc] = (new_val * color_mapping[ls])
        return(whole_mask)

    #this function is used to append new polygon k after its transformation
    #k est le 'Polygon' de clé k
    #feature_nb est l'indice du feature dans la feature_list
    def new_poly_append(k,feature_nb,df,poly_list):
        interior_coords = []
        #on teste si le 'Polygon' de clé k est du feature que l'on veut, si oui, on entre dans la boucle, si non, rien n'est ajouté à la poly_list
        if df['feattype'][k]['value']==features_list[feature_nb]:
            new_poly = df['geometry'][k]
            #we browse the features coming next in the list and modify the polygon if there is intersection
            #with polygons of those features
            for j,feature in enumerate(features_list[(feature_nb+1):]):
                df_polygon_geometry_feature = df[df["feattype"]== {'value': feature}].geometry
                for key2 in df_polygon_geometry_feature.keys():
                    if new_poly.intersects(df_polygon_geometry_feature[key2])==True:
                        new_poly = new_poly.difference(df_polygon_geometry_feature[key2])
            poly_list.append(new_poly)
        return(poly_list)

    def func_df(filepath):
        df = gpd.read_file(filepath)

        return(df)

    def polygon_df(filepath,features_list):
        df = gpd.read_file(filepath)
        #ON NE GARDE QUE LES FEATURES QUI SONT DU TYPE 'Polygon'
        df=df[df["geometry"].geom_type == 'Polygon']

        #LISTE DES NOUVEAUX POLYGONES, ROGNÉS ET MODIFIÉS QUE L'ON AJOUTERA DANS LA COLONNE 'geometry2'
        poly_list=[]

        #PARCOURS DANS L'ORDRE DES 'Polygon'
        for k in df.id.keys():
            l = len(poly_list)
            for feature_nb in range(len(features_list)):
                poly_list = new_poly_append(k,feature_nb,df,poly_list)
            #CE TEST SERT À SAVOIR SI NEW_POLY_APPEND A AJOUTÉ UN ELEMENT, SINON C'EST QUE LE POLYGON DE CLÉ K N'EST PAS UN FEATURE INTERESSANT   
            if l == len(poly_list):
                poly_list.append(df['geometry'][k])
        df['geometry2']=poly_list
        df['geometry2']=gpd.GeoDataFrame(df['geometry2'])
        return(df)

    # Fonctions de création de masques par lecture du GeoJson 
    # Idée: un petit objet inclut dans un autre doit avoir priorité sur l'affichage 

    def poly_from_utm(polygon, transform):
        #liste les points exterieurs du polygon
        poly_pts = []

        #liste les 'trous' dans ce polygon sous forme de tuple
        poly_holes = []

        exterior_coords = polygon.exterior.coords[:]
        interior_coords = []
        for interior in polygon.interiors:
            interior_coords.append(tuple(interior.coords[:]))

        for i in np.array(exterior_coords):
            poly_pts.append(~transform * tuple(i))

        for hole in np.array(interior_coords):
            hole_pts=[]
            for k in hole:

                hole_pts.append(tuple(~transform * tuple(k)))
            poly_holes.append(tuple(hole_pts))
        new_poly = Polygon(poly_pts,poly_holes)
        return new_poly

    # creating binary mask for field/not_filed segmentation.
    def create_mask(features_list,src,df):
        im_size = (src.meta['height'], src.meta['width'])
        whole_mask = np.zeros([src.meta['height'],src.meta['width']])

        for i,f in enumerate(features_list):
            poly_shp = []
            data_geom = df[df['feattype'] == {'value': f}].geometry2
            k = 0
            for g in data_geom.values:

                if g.geom_type == 'Polygon':
                    if not g.is_empty:

                        poly = poly_from_utm(g, src.meta['transform'])
                        poly_shp.append(poly)
                        poly_shp_polygon = poly_shp
                else:
                    for p in g:
                        poly = poly_from_utm(p, src.meta['transform'])
                        poly_shp.append(poly)

            if poly_shp:
                mask = rasterize(shapes=poly_shp,out_shape=im_size)*color_mapping[f]
                whole_mask = whole_mask + mask
        return whole_mask

    img_name = name_map+"_ortho.tif"
    directory = os.fsencode(path_GeoTIFF)

    poly_shp_test = []
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename==img_name:
            #print(path_GeoTIFF+"{}".format(filename))
            with rasterio.open(path_GeoTIFF+"{}".format(filename)) as src:
                img = src.read()
                meta = src.meta

                df = polygon_df(filepath,features_list)

                whole_mask= create_mask(features_list,src,df)

                whole_mask = draw_linestring(line_list,whole_mask,src.meta['transform'])

                # Just stores the different pixel values in the mask array, and print them.
                a=set([])

                for row in whole_mask:
                    for elt in row:
                        a.add(elt)
                print("Pixel values: "+str(a))

                
#                 # Just prints warning if two objects are superimposed
#                 a.remove(0)
#                 for elt in a:
#                     boolean=False
#                     for key in color_mapping:
#                         if(color_mapping[key] == elt):
#                             boolean=True;
#                     if not boolean:
#                         print("A superposition was detected !")

                print(whole_mask.shape)

                # Save numpy array into png mask 1 channel
                cv2.imwrite(outfolder + '/masktest_{}.png'.format(img_name.split('_')[0]),whole_mask)



Pixel values: {0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 7.0, 8.0, 9.0, 11.0, 12.0, 13.0, 14.0}
(9899, 23799)


## Segmentation, filtrage des images/masques contenant des pixels noirs et ajout éventuel d'un bruit statique

In [24]:
kernel_size = (256,256)
stride = (170,170)
add_static_noise = False
proba_noise = 0.5
path_masks='./global_model/v3/masks/'
path_imgs='./global_model/v3/imgs/'
os.makedirs(path_masks, exist_ok=True)
os.makedirs(path_imgs, exist_ok=True)

In [25]:
def gaussian_noise(size,mean = 0, sigma = 5): 
    return np.random.normal(mean, sigma, size).reshape(size[0],size[1],1)

In [26]:
path_GeoTIFF="./../GeoTIFF/"
path_big_masks="./global_model/big_masks"

list_imgs = os.listdir(path_GeoTIFF)
list_masks = os.listdir(path_big_masks)

data_dict = {}
for img in list_imgs: 
    name = img.split('_ortho')[0]
    data_dict[img] = [mask for mask in list_masks if name in mask ][0]

In [28]:
# Segment and add noise
for filename in data_dict: 
    filepath = os.path.join(path_big_masks,data_dict[filename])
    src = rasterio.open(path_GeoTIFF+"{}".format(filename))
    img = src.read().transpose((1,2,0))
    src.close()
    
    src=rasterio.open(path_big_masks+"/masktest_"+str(filename[:4])+".png")
    mask=src.read().transpose((1,2,0))
    src.close()
    
    for i in range((img.shape[0] - kernel_size[0])//stride[0] + 1):
        for j in range((img.shape[1] - kernel_size[1])//stride[1] + 1):
            img_window = img[i*stride[0]:i*stride[0]+kernel_size[0],j*stride[1]:j*stride[1]+kernel_size[1],:]
            mask_window = mask[i*stride[0]:i*stride[0]+kernel_size[0],j*stride[1]:j*stride[1]+kernel_size[1]]
            name_window = filename.split('.')[0] + '_{}_{}'.format(i,j)
            
            # Filters masks and images if they contain at least one black pixel
            if not np.sum(np.any(img_window==0)) and img_window.shape[0] == kernel_size[0] and img_window.shape[1] == kernel_size[1]:
                # Adds noise statically if add_noise is set to True
                if add_static_noise:
                    if np.random.uniform() < proba_noise: 
                        img_window = img_window + gaussian_noise(size = kernel_size).astype('int8') #/!\ negative value not supported
                # Generates image and mask
                cv2.imwrite(path_imgs+ name_window + '.png',img_window)
                cv2.imwrite(path_masks + name_window + '.png',mask_window)
    print('{} saved'.format(filename))

EDDW_ortho.tif saved
VTBD_ortho.tif saved
VHHH_ortho.tif saved
EIDW_ortho.tif saved
SBGR_ortho.tif saved
CYUL_ortho.tif saved
ZSDY_ortho.tif saved
LFPG_ortho.tif saved
CYEG_ortho.tif saved
LFOB_ortho.tif saved
RKPC_ortho.tif saved
LFMP_ortho.tif saved
LIRP_ortho.tif saved
LFBO_ortho.tif saved
ZBSJ_ortho.tif saved
DNMM_ortho.tif saved
LFMN_ortho.tif saved
FMEE_ortho.tif saved
KLAX_ortho.tif saved
OLBA_ortho.tif saved


## OLD (TOO Drastic, not used anymore): removes img/mask that only contain background

In [6]:
from PIL import Image
from os import listdir
from os.path import splitext
from glob import glob
from tqdm import tqdm

In [7]:
imgs_dir = './data/imgs/'
masks_dir = './data/masks/mask_'
ids = [splitext(file)[0] for file in listdir(imgs_dir)
            if not file.startswith('.')]

# Removes masks that only contain background (borders)
rm_files = 0
for id in tqdm(ids):
    idx = id
    mask_file = glob(masks_dir + idx + '.png')
    img_file = glob(imgs_dir + idx + '.png')
    mask = Image.open(mask_file[0])

    if np.unique(np.array(mask)).shape[0] == 1:
        os.remove(img_file[0])
        mask.close()
        os.remove(mask_file[0])
        rm_files +=1

print('DONE! Removed files: ',rm_files)

100%|██████████| 34213/34213 [00:36<00:00, 936.46it/s]

DONE! Removed files:  21038


In [27]:
A= [1,1,0]
np.any(A==0)

False